In [3]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, LineString, shape
# !pip install passiogo
import passiogo




In [70]:
#uchicago
system = passiogo.getSystemFromID(1068)

rows = []

#get all the stops and their positions
for stop in system.getStops():
    stop_dict = stop.__dict__
    for route, stop_list in stop_dict['routesAndPositions'].items():
        for stop_pos in stop_list:
            row = {'route_id':route, 'stop_id':stop_dict['id'], 'stop_name':stop_dict['name'], 'stop_position': stop_pos, 'stop_lat':stop_dict['latitude'], 'stop_lon':stop_dict['longitude']}
            rows.append(row)
        
df_stops = pd.DataFrame(rows)

#grab some route metadata
routes = system.getRoutes()
rows = []
for route in routes:
    route_dict = route.__dict__
    row = {'route_id':route_dict['myid'], 'route_name':route_dict['name'], 'outdated':route_dict['outdated']}
    rows.append(row)

df_routes = pd.DataFrame(rows)
   
#combine stops and route metadata
df = df_stops.merge(df_routes, on='route_id')
df.sort_values(['route_id', 'stop_position'], inplace=True)

#zip the coordinates into a point object and convert to a GeoData Frame
geometry = [Point(xy) for xy in zip(df.stop_lon, df.stop_lat)]
geo_df = gpd.GeoDataFrame(df, geometry=geometry)

# remove 4009 since it just has one point
geo_df = geo_df[geo_df.route_id != '4009']

geo_df2 = geo_df.groupby(['route_id'])['geometry'].apply(lambda x: LineString(x.tolist())).reset_index()
geo_df2 = gpd.GeoDataFrame(geo_df2, geometry='geometry', crs="epsg:4326")

In [ ]:
geo_df2.explore(column='route_id', cmap='tab20', figsize=(20, 20))

In [56]:
df.sort_values(['route_id', 'stop_position'])

,route_id,stop_id,stop_name,stop_position,stop_lat,stop_lon,route_name,outdated
1,38601,8612,Goldblatt Pavilion,0,41.787743,-87.603607,Friend Center/Metra,0
3,38601,8613,Wyler Pavilion,1,41.788802,-87.604871,Friend Center/Metra,0
4,38601,8615,Bernard Mitchell Hospital,2,41.789429,-87.604871,Friend Center/Metra,0
5,38601,8616,58th Street & Drexel,3,41.789507,-87.603924,Friend Center/Metra,0
6,38601,8617,Comer Children's Hospital,4,41.790505,-87.603647,Friend Center/Metra,0
...,...,...,...,...,...,...,...,...
308,ctaX4,cta76,Michigan & Jackson (CTA),32,41.878143,-87.624199,Cottage Grove Express (CTA),0
338,ctaX4,cta15241,Michigan & Monroe (CTA),33,41.880672,-87.624192,Cottage Grove Express (CTA),0
316,ctaX4,cta2188,Michigan & Madison (CTA),34,41.882375,-87.624211,Cottage Grove Express (CTA),0
332,ctaX4,cta14482,Randolph & Michigan (CTA),35,41.884293,-87.623626,Cottage Grove Express (CTA),0


In [4]:
fname = "property_years_extended_11_17_24.geojson"

df_gpd = gpd.read_file(fname)


In [5]:
df_gpd[df_gpd['currently_exists'] & ~df_gpd['currently_owned']]

,osm_id,custom_id,name,address,year_start,year_end,currently_owned,years_confirmed,quad_flag,recent_residential_sales,charter_schools_flag,commercial_buildings_flag,currently_exists,geometry
3,211224089,None,1310-16 East Hyde Park Boulevard (1),1310-16 East Hyde Park Boulevard,2000.0,2020.0,False,False,False,25.0,False,False,True,"MULTIPOLYGON (((-87.59462 41.80260, -87.59463 ..."
4,211224094,None,1310-16 East Hyde Park Boulevard (2),1310-16 East Hyde Park Boulevard,2000.0,2020.0,False,False,False,25.0,False,False,True,"MULTIPOLYGON (((-87.59433 41.80279, -87.59433 ..."
5,211224110,None,1318-24 East Hyde Park Boulevard (1),1318-24 East Hyde Park Boulevard,2000.0,2020.0,False,False,False,26.0,False,False,True,"MULTIPOLYGON (((-87.59429 41.80260, -87.59431 ..."
6,211224105,None,1318-24 East Hyde Park Boulevard (2),1318-24 East Hyde Park Boulevard,2000.0,2020.0,False,False,False,26.0,False,False,True,"MULTIPOLYGON (((-87.59402 41.80279, -87.59401 ..."
7,150150369,None,1321 East 57th Street,1321 East 57th Street,2009.0,2021.0,False,False,False,2.0,False,True,True,"MULTIPOLYGON (((-87.59389 41.79138, -87.59396 ..."
8,226829155,None,1323 East 57th Street,1323 East 57th Street,2009.0,2021.0,False,False,False,2.0,False,True,True,"MULTIPOLYGON (((-87.59389 41.79117, -87.59379 ..."
9,226829154,None,1327 East 57th Street,1327 East 57th Street,2011.0,2021.0,False,False,False,0.0,False,True,True,"MULTIPOLYGON (((-87.59379 41.79117, -87.59379 ..."
10,226828772,None,1329-33 East 57th Street,1329-33 East 57th Street,2011.0,2021.0,False,False,False,0.0,False,True,True,"MULTIPOLYGON (((-87.59370 41.79127, -87.59364 ..."
11,None,38,1330 East 53rd Street,1330 East 53rd Street,2010.0,2020.0,False,False,False,-1.0,False,True,True,"MULTIPOLYGON (((-87.59425 41.80001, -87.59424 ..."
13,211224123,None,1334-40 East Hyde Park Boulevard (1),1334-40 East Hyde Park Boulevard,2000.0,2020.0,False,False,False,28.0,False,False,True,"MULTIPOLYGON (((-87.59366 41.80261, -87.59368 ..."


In [3]:
df_pd = pd.DataFrame(df_gpd)

In [4]:
df_pd

,osm_id,custom_id,name,address,year_start,year_end,currently_owned,geometry
0,445414332.0,NaN,1330 East 53rd Street,1330 East 53rd Street,2016.0,2024.0,True,"POLYGON ((-87.59425 41.80000, -87.59424 41.799..."
1,211224202.0,NaN,1447-51 East 53rd Street,1447-51 East 53rd Street,2011.0,2024.0,True,"POLYGON ((-87.58977 41.79943, -87.58997 41.799..."
2,143139313.0,NaN,Hyde Park Art Center,5020 South Cornell Avenue,1939.0,2024.0,True,"POLYGON ((-87.58695 41.80310, -87.58660 41.803..."
3,NaN,27.0,Hyde Park Arts Center Parking Lot,5020 South Cornell Avenue,1939.0,2024.0,True,"POLYGON ((-87.58697 41.80417, -87.58697 41.803..."
4,145510725.0,NaN,6022-24 South Drexel Avenue,6022-24 South Drexel Avenue,1995.0,2024.0,True,"POLYGON ((-87.60510 41.78518, -87.60514 41.785..."
...,...,...,...,...,...,...,...,...
185,505873479.0,NaN,UChicago Charter Woodlawn Campus,6420 South University Avenue,2006.0,2024.0,True,"POLYGON ((-87.59870 41.78005, -87.59834 41.780..."
186,156048759.0,NaN,Blackstone Hall,5748 South Blackstone Avenue,1930.0,2024.0,True,"POLYGON ((-87.59039 41.79015, -87.59043 41.790..."
187,144479783.0,NaN,Broadview Hall,5540 South Hyde Park Boulevard,1927.0,2024.0,True,"POLYGON ((-87.58461 41.79400, -87.58415 41.794..."
188,144008990.0,NaN,Maclean Hall,5445 South Ingleside Avenue,1991.0,2024.0,True,"POLYGON ((-87.60260 41.79687, -87.60259 41.796..."


In [17]:
bins = [1890,1930,1960,1990,2025]
labels = ["1890-1930", "1930-1960", "1960-1990","1990-2025"]

df_pd['era'] = pd.cut(x=df_pd['year_start'], bins=bins, labels=labels)

df_pd['era'] = df_pd['era'].astype(str)

In [18]:
with_eras = gpd.GeoDataFrame(df_pd, crs="EPSG:4326")


In [19]:
with_eras.to_file("with_era.geojson", driver="GeoJSON")  

In [5]:
import requests
import json

url = 'https://api.maptiler.com/maps/dataviz/style.json?key=aXEOZFRxlPWOaJqsWprN'
r = requests.get(url)
tiles = r.json()    

with open("tiles.json", "w") as outfile: 
    json.dump(tiles, outfile)


AttributeError: 'dict' object has no attribute 'to_json'